In [ ]:
import os
import random
import re
import string

import nltk
import numpy as np
import pandas as pd

from gensim.models import Word2Vec

from collections import Counter

from nltk import word_tokenize,sent_tokenize

from nltk.corpus import stopwords

from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score
nltk.download('punkt')
nltk.download("stopwords")

SEED = 42
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)

C:\Users\Greeshma Theresa\Anaconda\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to C:\Users\Greeshma
[nltk_data]     Theresa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Greeshma
[nltk_data]     Theresa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_json("C:\\Users\\Greeshma Theresa\\Downloads\\data_small.json")

In [ ]:
data.columns

Index(['user_message_id', 'sender', 'message', 'timestamp', 'created_at',
       'user_id', 'user_subscription_id', 'okay'],
      dtype='object')

In [ ]:
data.head(100)

,user_message_id,sender,message,timestamp,created_at,user_id,user_subscription_id,okay
0,4317860,ADSBUXIN,"Hi, our siren is at home and safe and we hope ...",2020-03-25 07:33:39.442,2021-09-16 04:43:11.234,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
1,4317861,TMKOTAKB,"Due to COVID-19, some of the banking services ...",2020-03-25 06:34:38.445,2021-09-16 04:43:11.236,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
2,4317862,ADMONVEW,Youre a few steps away from completing your Cr...,2020-03-25 06:04:45.183,2021-09-16 04:43:11.238,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
3,4317863,VKEZHOME,"Hi Niraj, a special Home Loan offer for you: \...",2020-03-25 06:02:46.155,2021-09-16 04:43:11.241,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
4,4317864,JXJIOMNY,"Dear JioMoney Customer, Pay from Home using My...",2020-03-25 05:54:36.521,2021-09-16 04:43:11.244,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
...,...,...,...,...,...,...,...,...
95,4317954,BHCARSPA,"Car deep cleaning, polishing & waxing at your ...",2020-03-20 06:45:24.267,2021-09-16 04:43:11.468,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
96,4317955,TXCRWNIT,Here's Your Chance To Earn Rs.1250 PayTM! Refe...,2020-03-20 06:38:06.699,2021-09-16 04:43:11.470,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
97,4317956,ADMONVEW,Youre a few steps away from completing your Cr...,2020-03-20 06:11:27.013,2021-09-16 04:43:11.473,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
98,4317957,ADPAYZAP,You've won upto Rs. 250 Amazon voucher!\nClaim...,2020-03-20 06:09:25.513,2021-09-16 04:43:11.475,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True


In [ ]:
df_small=data.head(100000)

Used a data of 100000 entries for implementing K means algorithm

In [ ]:
def clean_text(text, tokenizer, stopwords):
    """Pre-process text and generate tokens

    Args:
        text: Text to tokenize.

    Returns:
        Tokenized text.
    """
    text = str(text).lower()  # Lowercase words
    text = re.sub(r"\[(.*?)\]", "", text)  # Remove [+XYZ chars] in content
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces in content
    text = re.sub(r"\w+…|…", "", text)  # Remove ellipsis (and last word)
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text)  # Replace dash between words
    text = re.sub(
        f"[{re.escape(string.punctuation)}]", "", text
    )  # Remove punctuation

    tokens = tokenizer(text)  # Get tokens from text
    tokens = [t for t in tokens if not t in stopwords]  # Remove stopwords
    tokens = ["" if t.isdigit() else t for t in tokens]  # Remove digits
    tokens = [t for t in tokens if len(t) > 1]  # Remove short tokens
    return tokens

In [ ]:
custom_stopwords = set(stopwords.words("english"))
text_columns = ["message"]

df = df_small.copy()
df["message"] = df["message"].fillna("")
df["message"] = df["message"].astype(str)

# Create text column based on title, description, and content
df["text"] = df[text_columns].apply(lambda x: " | ".join(x), axis=1)
df["tokens"] = df["text"].map(lambda x: clean_text(x, word_tokenize, custom_stopwords))

# Remove duplicated after preprocessing
_, idx = np.unique(df["tokens"], return_index=True)
df = df.iloc[idx, :]

# Remove empty values
df = df.loc[df.tokens.map(lambda x: len(x) > 0), ["text", "tokens"]]

print(f"Original dataframe: {data.shape}")
print(f"Pre-processed dataframe: {df.shape}")

Original dataframe: (1898953, 8)
Pre-processed dataframe: (65370, 2)


Output: Original dataframe: (1898953, 8)
        Pre-processed dataframe: (65370, 2)

In [ ]:
docs = df["text"].values
tokenized_docs = df["tokens"].values
vocab = Counter()
for token in tokenized_docs:
    vocab.update(token)

In [ ]:
def vectorize(list_of_docs, model):
    """Generate vectors for list of documents using a Word Embedding

    Args:
        list_of_docs: List of documents
        model: Gensim's Word Embedding

    Returns:
        List of document vectors
    """
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features


In [ ]:
model = Word2Vec(sentences=tokenized_docs, vector_size=100, workers=1, seed=42)

In [ ]:
model.wv.most_similar("bal")

[('sakcash', 0.7355659008026123),
 ('balrs', 0.7053314447402954),
 ('cdm', 0.7047985792160034),
 ('lmt', 0.6905398368835449),
 ('bk', 0.6885620355606079),
 ('20infoamazon', 0.6490204930305481),
 ('noida', 0.6440922617912292),
 ('cashrs1', 0.6298109889030457),
 ('xx8139', 0.6293550729751587),
 ('one97', 0.6277592182159424)]

In [ ]:
vectorized_docs = vectorize(tokenized_docs, model=model)
len(vectorized_docs), len(vectorized_docs[0])

(65370, 100)

output: (65370, 100)

In [ ]:
def mbkmeans_clusters(X, k, mb=10000, print_silhouette_values=False):
    """Generate clusters.

    Args:
        X: Matrix of features.
        k: Number of clusters.
        mb: Size of mini-batches. Defaults to 500.
        print_silhouette_values: Print silhouette values per cluster.

    Returns:
        Trained clustering model and labels based on X.
    """
    km = MiniBatchKMeans(n_clusters=k, batch_size=mb).fit(X)
    print(f"For n_clusters = {k}")
    print(f"Silhouette coefficient: {silhouette_score(X, km.labels_):0.2f}")
    print(f"Inertia:{km.inertia_}")

    if print_silhouette_values:
        sample_silhouette_values = silhouette_samples(X, km.labels_)
        print(f"Silhouette values:")
        silhouette_values = []
        for i in range(k):
            cluster_silhouette_values = sample_silhouette_values[km.labels_ == i]
            silhouette_values.append(
                (
                    i,
                    cluster_silhouette_values.shape[0],
                    cluster_silhouette_values.mean(),
                    cluster_silhouette_values.min(),
                    cluster_silhouette_values.max(),
                )
            )
        silhouette_values = sorted(
            silhouette_values, key=lambda tup: tup[2], reverse=True
        )
        for s in silhouette_values:
            print(
                f"    Cluster {s[0]}: Size:{s[1]} | Avg:{s[2]:.2f} | Min:{s[3]:.2f} | Max: {s[4]:.2f}"
            )
    return km, km.labels_

In [ ]:

clustering, cluster_labels = mbkmeans_clusters(X=vectorized_docs, k=3, print_silhouette_values=True)
df_clusters = pd.DataFrame({
    "text": docs,
    "tokens": [" ".join(text) for text in tokenized_docs],
    "cluster": cluster_labels
})

For n_clusters = 3
Silhouette coefficient: 0.29
Inertia:42098.01776055332
Silhouette values:
    Cluster 1: Size:2362 | Avg:0.46 | Min:-0.02 | Max: 0.65
    Cluster 0: Size:8139 | Avg:0.26 | Min:-0.04 | Max: 0.46
    Cluster 2: Size:2547 | Avg:0.23 | Min:-0.01 | Max: 0.44


In [ ]:
print("Top terms per cluster (based on centroids):")
for i in range(3):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=5)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")

Top terms per cluster (based on centroids):
Cluster 0: transacting making httpsszzlio05qfet october unlocked 
Cluster 1: debited inr dbs vpa neft 
Cluster 2: low od needed hassle insta 


In [ ]:
#output

Top terms per cluster (based on centroids):
Cluster 0: transacting making httpsszzlio05qfet october unlocked 
Cluster 1: debited inr dbs vpa neft 
Cluster 2: low od needed hassle insta 


Cluster 1 consists of transactional msgs and they are filtered using k means


In [ ]:

test_cluster = 1
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:5]:
    print(docs[d])
    print("-------------")

Dear Customer, your ICICI Bank Account XX481 has been credited with INR 7,198.19 on 16-Sep-21. Info:EBA*BSE N 202122114*20210916082932. The Available Balance is INR 9,409.75.
-------------
Dear Customer, your DBS account no ********0809 is credited with INR 200 on 06-09-2021 and is subject to clearance. Current Balance is INR 231.4.
-------------
Dear Customer, ICICI Bank Account XX481 is debited with INR 6,907.50 on 03-Sep-21. Info: EBA*BSE N 202. The Available Balance is INR 24,864.78. Call 18002662 for dispute or SMS BLOCK 481 to 9215676766
-------------
Dear Customer, ICICI Bank Account XX481 is debited with INR 5,981.50 on 08-Sep-21. Info: EBA*NSE N 202. The Available Balance is INR 9,073.80. Call 18002662 for dispute or SMS BLOCK 481 to 9215676766
-------------
Your VPA nirajsachdev@dbs linked to DBS Bank India Limited  a/c no. ********0809 is debited for ₹249.0 and the amount is credited to VPA nearbuygroup@paytm (UPI Ref no 107819013751).
-------------


output:

Dear Customer, your ICICI Bank Account XX481 has been credited with INR 7,198.19 on 16-Sep-21. Info:EBA*BSE N 202122114*20210916082932. The Available Balance is INR 9,409.75.
-------------
Dear Customer, your DBS account no ********0809 is credited with INR 200 on 06-09-2021 and is subject to clearance. Current Balance is INR 231.4.
-------------
Dear Customer, ICICI Bank Account XX481 is debited with INR 6,907.50 on 03-Sep-21. Info: EBA*BSE N 202. The Available Balance is INR 24,864.78. Call 18002662 for dispute or SMS BLOCK 481 to 9215676766
-------------
Dear Customer, ICICI Bank Account XX481 is debited with INR 5,981.50 on 08-Sep-21. Info: EBA*NSE N 202. The Available Balance is INR 9,073.80. Call 18002662 for dispute or SMS BLOCK 481 to 9215676766
-------------
Your VPA nirajsachdev@dbs linked to DBS Bank India Limited  a/c no. ********0809 is debited for ₹249.0 and the amount is credited to VPA nearbuygroup@paytm (UPI Ref no 107819013751).
-------------






Expense and earning of an user with user_id='40ec6ba2-af1b-4037-9e01-3d108faa937b'





In [ ]:
dk=data[data["user_id"]=='40ec6ba2-af1b-4037-9e01-3d108faa937b']

In [ ]:
dk.shape

(17903, 8)

In [ ]:
dk

,user_message_id,sender,message,timestamp,created_at,user_id,user_subscription_id,okay
0,4317860,ADSBUXIN,"Hi, our siren is at home and safe and we hope ...",2020-03-25 07:33:39.442,2021-09-16 04:43:11.234,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
1,4317861,TMKOTAKB,"Due to COVID-19, some of the banking services ...",2020-03-25 06:34:38.445,2021-09-16 04:43:11.236,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
2,4317862,ADMONVEW,Youre a few steps away from completing your Cr...,2020-03-25 06:04:45.183,2021-09-16 04:43:11.238,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
3,4317863,VKEZHOME,"Hi Niraj, a special Home Loan offer for you: \...",2020-03-25 06:02:46.155,2021-09-16 04:43:11.241,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
4,4317864,JXJIOMNY,"Dear JioMoney Customer, Pay from Home using My...",2020-03-25 05:54:36.521,2021-09-16 04:43:11.244,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
...,...,...,...,...,...,...,...,...
5898191,4317854,TXPASBAZ,"Niraj, Your latest Experian credit score needs...",2020-03-26 06:12:21.604,2021-09-16 04:43:11.216,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
5898192,4317855,AMWYNKED,Your free Wynk Premium subscription is now act...,2020-03-26 05:48:41.942,2021-09-16 04:43:11.218,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
5898193,4317856,AMARTLTV,Your FREE access to Airtel Xstream is now acti...,2020-03-25 23:51:19.650,2021-09-16 04:43:11.224,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True
5898194,4317857,ADEPFOHO,"Dear 101513602369, your passbook balance again...",2020-03-25 11:55:57.718,2021-09-16 04:43:11.226,40ec6ba2-af1b-4037-9e01-3d108faa937b,NaN,True







Separating transactinal msgs for data with a particular user with user_id='40ec6ba2-af1b-4037-9e01-3d108faa937b'







In [ ]:
custom_stopwords = set(stopwords.words("english"))
text_columns = ["message"]

df = dk.copy()
df["message"] = df["message"].fillna("")
df["message"] = df["message"].astype(str)

# Create text column based on title, description, and content
df["text"] = df[text_columns].apply(lambda x: " | ".join(x), axis=1)
df["tokens"] = df["text"].map(lambda x: clean_text(x, word_tokenize, custom_stopwords))

# Remove duplicated after preprocessing
_, idx = np.unique(df["tokens"], return_index=True)
df = df.iloc[idx, :]

# Remove empty values
df = df.loc[df.tokens.map(lambda x: len(x) > 0), ["text", "tokens"]]

print(f"Original dataframe: {data.shape}")
print(f"Pre-processed dataframe: {df.shape}")

Original dataframe: (1898953, 8)
Pre-processed dataframe: (13048, 2)


output:Original dataframe: (1898953, 8)
Pre-processed dataframe: (13048, 2)

In [ ]:
docs = df["text"].values
tokenized_docs = df["tokens"].values
vocab = Counter()
for token in tokenized_docs:
    vocab.update(token)

In [ ]:
model = Word2Vec(sentences=tokenized_docs, vector_size=100, workers=1, seed=42)

In [ ]:
vectorized_docs = vectorize(tokenized_docs, model=model)
len(vectorized_docs), len(vectorized_docs[0])


(13048, 100)

output:(13048, 100)

In [ ]:
clustering, cluster_labels = mbkmeans_clusters(X=vectorized_docs, k=3, print_silhouette_values=True)
df_clusters = pd.DataFrame({
    "text": docs,
    "tokens": [" ".join(text) for text in tokenized_docs],
    "cluster": cluster_labels
})

For n_clusters = 3
Silhouette coefficient: 0.25
Inertia:41578.341939507634
Silhouette values:
    Cluster 1: Size:2260 | Avg:0.46 | Min:-0.03 | Max: 0.65
    Cluster 2: Size:5512 | Avg:0.26 | Min:0.01 | Max: 0.44
    Cluster 0: Size:5276 | Avg:0.15 | Min:-0.09 | Max: 0.35


Output: For n_clusters = 3
Silhouette coefficient: 0.24
Inertia:41642.0371021783
Silhouette values:
    Cluster 1: Size:2154 | Avg:0.47 | Min:0.00 | Max: 0.66
    Cluster 2: Size:4994 | Avg:0.24 | Min:-0.02 | Max: 0.42
    Cluster 0: Size:5900 | Avg:0.17 | Min:-0.08 | Max: 0.36

Cluster 1 consists of transactional msgs

In [ ]:
print("Top terms per cluster (based on centroids):")
for i in range(3):
    tokens_per_cluster = ""
    most_representative = model.wv.most_similar(positive=[clustering.cluster_centers_[i]], topn=5)
    for t in most_representative:
        tokens_per_cluster += f"{t[0]} "
    print(f"Cluster {i}: {tokens_per_cluster}")

Top terms per cluster (based on centroids):
Cluster 0: less additional offertc od rs22241lakh 
Cluster 1: debited inr dbs vpa xxxxxxx1541 
Cluster 2: rs559989 rs565589 completed accept wise 


 Cluster 0 : Offers
 Cluster 1 : Trasactions
 Cluster 2 : Info
 15 Transactional messsages( Cluster =2 )for user_id='40ec6ba2-af1b-4037-9e01-3d108faa937b'

In [ ]:

test_cluster = 2
most_representative_docs = np.argsort(
    np.linalg.norm(vectorized_docs - clustering.cluster_centers_[test_cluster], axis=1)
)
for d in most_representative_docs[:15]:
    print(docs[d])
    print("-------------")

Dear Customer, JioMoney will be available ONLY in MyJio from 15th July, 2020. For easy Jio Recharges, visit MyJio for an enhanced payment experience. To begin, tap here https://bit.ly/2Y70utq
-------------
Dear Customer, due to COVID-19, we are short-staffed at our Customer Contact Centre and will be available from 10 a.m. to 6 p.m. We encourage you to use Mobile/Net Banking for 24x7 access to your accounts. To know more, click here bit.ly/KotakCECtimings
-------------
A maintenance charge of Rs.20(incl. GST) will be applied on your Airtel Money Wallet as you have not used it for the last 6 months. If you do not wish to have this charge applied, simply use your wallet for any transaction before 20th July. Please do refer https://i.airtel.in/Terms or call 8800688006 for details.
-------------
Dear NICB Customer, due to lockdown conditions in most of the areas we are working with reduced capacity and have revised our banking hours temporarily. Branches will now operate from 10:00 AM to 0

Each transactional msg is split into a list of strings using split function

In [ ]:
for d in most_representative_docs[:15]:
    res=docs[d].split()
    
    print(res)
    print(".........................")

['Dear', 'Customer,', 'your', 'ICICI', 'Bank', 'Account', 'XX481', 'has', 'been', 'credited', 'with', 'INR', '7,198.19', 'on', '16-Sep-21.', 'Info:EBA*BSE', 'N', '202122114*20210916082932.', 'The', 'Available', 'Balance', 'is', 'INR', '9,409.75.']
.........................
['Dear', 'Customer,', 'ICICI', 'Bank', 'Account', 'XX481', 'is', 'debited', 'with', 'INR', '6,907.50', 'on', '03-Sep-21.', 'Info:', 'EBA*BSE', 'N', '202.', 'The', 'Available', 'Balance', 'is', 'INR', '24,864.78.', 'Call', '18002662', 'for', 'dispute', 'or', 'SMS', 'BLOCK', '481', 'to', '9215676766']
.........................
['Dear', 'Customer,', 'ICICI', 'Bank', 'Account', 'XX481', 'is', 'debited', 'with', 'INR', '5,981.50', 'on', '08-Sep-21.', 'Info:', 'EBA*NSE', 'N', '202.', 'The', 'Available', 'Balance', 'is', 'INR', '9,073.80.', 'Call', '18002662', 'for', 'dispute', 'or', 'SMS', 'BLOCK', '481', 'to', '9215676766']
.........................
['Dear', 'Customer,', 'Account', 'XXX481', 'is', 'credited', 'with', 'INR

User balance for user_id='40ec6ba2-af1b-4037-9e01-3d108faa937b'

In [ ]:
cred_t=0
for d in most_representative_docs[:]:
    res=docs[d].split()
    #print
    #b1=
    for i in range(len(res)): 
        if res[i]=="credited":
             for j in range(len(res)):
                if res[j]=="INR":
                    cred_t+=float(res[j+1].replace(",", ""))
                    #print(cred_t)
                    break
                elif res[j]=="Rs":
                    cred_t+=float(res[j+1].replace(",", ""))
                    break
              
        elif res[i]=="debited":
              for j in range(len(res)):
                if res[j]=="INR":
                    cred_t-=float(res[j+1].replace(",", ""))
                    #print(cred_t)
                    break
                elif res[j]=="Rs":
                    cred_t-=float(res[j+1].replace(",", ""))
                    break
print(cred_t)   
            
      

806558.96


Output:806558.96

In [ ]:
#Total credited amount for user_id='40ec6ba2-af1b-4037-9e01-3d108faa937b'

In [ ]:
cred_t=0
for d in most_representative_docs[:]:
    res=docs[d].split()
    #print
    #b1=
    for i in range(len(res)): 
        if res[i]=="credited":
             for j in range(len(res)):
                if res[j]=="INR":
                    cred_t+=float(res[j+1].replace(",", ""))
                    #print(cred_t)
                    break
                elif res[j]=="Rs":
                    cred_t+=float(res[j+1].replace(",", ""))
                    break
print(cred_t)             

1805521.81


Output:1805521.81

In [ ]:
#Total debited amount for user_id='40ec6ba2-af1b-4037-9e01-3d108faa937b'

In [ ]:
cred_t=0
for d in most_representative_docs[:]:
    res=docs[d].split()
    #print
    #b1=
    for i in range(len(res)): 
        if res[i]=="debited":
             for j in range(len(res)):
                if res[j]=="INR":
                    cred_t+=float(res[j+1].replace(",", ""))
                    #print(cred_t)
                    break
                elif res[j]=="Rs":
                    cred_t+=float(res[j+1].replace(",", ""))
                    break
print(cred_t)              

998962.85


Output:998962.85